# Hackathon #4

Written by Eleanor Quint

Topics: 
- Convolutional layers
- Pooling layers

This is all setup in a IPython notebook so you can run any code you want to experiment with. Feel free to edit any cell, or add some to run your own code.

In [ ]:
# We'll start with our library imports...
from __future__ import print_function

import numpy as np                 # to use numpy arrays
import tensorflow as tf            # to specify and run computation graphs
import tensorflow_datasets as tfds # to load training data
import matplotlib.pyplot as plt    # to visualize data and draw plots
from tqdm import tqdm              # to track progress of loops

DATA_DIR = './tensorflow-datasets/'

In this hackathon, we'll be using convolutional layers, which are uniquely suited to working with image data. We'll use a dataset which is much more challenging if you're only using dense layers, [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html).

In [ ]:
ds = tfds.load('cifar10', shuffle_files=True, data_dir=DATA_DIR) # this loads a dict with the datasets

# We can create an iterator from each dataset
# This one iterates through the train data, shuffling and minibatching by 32
train_ds = ds['train'].shuffle(1024).batch(32)

# Looping through the iterator, each batch is a dict
for batch in train_ds:
    print("data shape:", batch['image'].shape)
    print("label:", batch['label'])
    break

# visualize some of the data, pick randomly every time this cell is run
idx = np.random.randint(batch['image'].shape[0])
print("An image looks like this:")
imgplot = plt.imshow(batch['image'][idx])

## Convolutional Layers

TensorFlow implements the convolutional layer with [`tf.keras.layers.Conv2D`](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Conv2D). The function that instantiates the layer has two required arguments: number of filters and filter size. The number of filters will be equal to the number of channels in the output.

Important to keep in mind is:

1. Input data should be 4-dimensional with shape (batch, height, width, channels), unless the data_format argument is specified.
2. Padding is `'valid'` by default, meaning that only filters which lie fully within the input image will be kept. This will make the resulting image slightly smaller than the input. Use `padding='same'` if image size should be preserved.
3. Specifying `strides=n` for some n > 1 will result in an output image multiplicatively smaller than the input by a factor of n. Make sure that n isn't greater than the filter size unless you intend to completely ignore part of the input.

Let's specify a convolutional classifier.

In [ ]:
hidden_1 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation=tf.nn.relu, name='hidden_1')
hidden_2 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation=tf.nn.relu, name='hidden_2')
flatten = tf.keras.layers.Flatten()
output = tf.keras.layers.Dense(10)
conv_classifier = tf.keras.Sequential([hidden_1, hidden_2, flatten, output])

This architecture of a convolutional stack followed by dense layers for classification is pretty typical. It has the major advantages of making spacially local transformations to the image, followed by a global transformation which makes the final classification decision. You might notice as well that the number of filters goes up after each convolution. This is to allow higher layers to learn more specific patterns, so we require more of them than the general patterns of the earlier layers.

The number of parameters in each convolution layer can be calculated as `filter_height * filter_width * in_channels * output_channels`, as opposed to dense layers which have `input_size * output_size` parameters. For example, if we're working with CIFAR images, a first layer 3x3 convolution with 32 filters will have `3 * 3 * 3 * 32 = 864` parameters, compared to a dense layer with 32 neurons' `(32 * 32 * 3) * 32 = 98304` parameters. This is a factor of ~114 decrease, significantly smaller!

We can use a function of `tf.keras.Sequential` to see more information about the network, but only after the model has been built.

In [ ]:
# Run some data through the network to initialize it
for batch in train_ds:
    # data is uint8 by default, so we have to cast it
    conv_classifier(tf.cast(batch['image'], tf.float32))
    break
conv_classifier.summary()

Notice that the number of channels in each output shape is equal to the number of filters in the channel. Notice also that the Flatten layer changes the shape of the data to only having two dimensions, this allows us to feed it to the Dense layer in the usual way.

Let's re-create the network which we used to classify MNIST from Hackathon 2.

In [ ]:
# Create the model
dense_classifier = tf.keras.Sequential([tf.keras.layers.Flatten(), tf.keras.layers.Dense(200), tf.keras.layers.Dense(10)])

# Initialize the model
for batch in train_ds:
    # data is uint8 by default, so we have to cast it
    dense_classifier(tf.cast(batch['image'], tf.float32))
    break

dense_classifier.summary()

This densely connected network has fewer parameters than the convolutional network! If we compare them, the one dense layer in the convolutional network has more paramters than the entire dense network. This is because the representation grows as it passes through the conv network, which is undesirable. We can handle this by summarizing the data with pooling layers. We'll add max pool layers which act similarly to conv layers, but rather than computing a function with a filter, it selects the pixel with the highest value in the window and ignores the rest.

Let's respecify a larger conv network than the above, but this time with max pooling layers:

In [ ]:
hidden_1 = tf.keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', activation=tf.nn.relu, name='hidden_1')
hidden_2 = tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', activation=tf.nn.relu, name='hidden_2')
pool_1 = tf.keras.layers.MaxPool2D(padding='same')
hidden_3 = tf.keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', activation=tf.nn.relu, name='hidden_3')
hidden_4 = tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding='same', activation=tf.nn.relu, name='hidden_4')
pool_2 = tf.keras.layers.MaxPool2D(padding='same')
flatten = tf.keras.layers.Flatten()
output = tf.keras.layers.Dense(10)
conv_classifier = tf.keras.Sequential([hidden_1, hidden_2, pool_1, hidden_3, hidden_4, pool_2, flatten, output])

# Run some data through the network to initialize it
for batch in train_ds:
    # data is uint8 by default, so we have to cast it
    conv_classifier(tf.cast(batch['image'], tf.float32))
    break
conv_classifier.summary()

With pooling we've created a much bigger network with fewer parameters. Another way to reduce the size of the data is by setting the `stride` parameter of conv layers to be greater than one (typically two). This will have a similar effect to pooling in reducing the size of the representation by a factor of 4.

### Homework

Re-write your code from hackathon 2 to use convolutional layers and add code to plot a confusion matrix on the validation data.

Specifically, write code to calculate a confusion matrix of the model output on the validation data, and compare to the true labels to calculate a confusion matrix with [tf.math.confusion_matrix](https://www.tensorflow.org/api_docs/python/tf/math/confusion_matrix). (For the inexperienced, [what is a confusion matrix?](https://en.wikipedia.org/wiki/Confusion_matrix)) Use the code example from [scikit-learn](https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html) to help visualise the confusion matrix if you'd like as well.

On Canvas, submit your python code in a `.py` and your confusion matrix in a `.png` or `.txt`.

I'm expecting this to take about an hour (or less if you're experienced). Feel free to use any code from this or previous hackathons. If you don't understand how to do any part of this or if it's taking you longer than that, please let me know in office hours or by email (both can be found on the syllabus). I'm also happy to discuss if you just want to ask more questions about anything in this notebook!

### Coda

#### Convolutional Filters from the First Layer of ImageNet

Interestingly, automatically learned filters often closely resemble [Gabor Filters](https://en.wikipedia.org/wiki/Gabor_filter). The first layer of the original ImageNet network learned the following filters:

![](http://smerity.com/media/images/articles/2016/imagenet_conv_kernels.png "Convolutional Filters from the First Layer of ImageNet")

#### [Visualizing Convolutional Features](https://distill.pub/2017/feature-visualization/)